In [1]:
DATA_LINK = "https://docs.google.com/spreadsheets/d/1LYYyjZtW7cLXMN5mTh3-K1pBRn6ncA_T"
APPEND_TO_LINK = "/gviz/tq?tqx=out:html&tq&gid=1"
TABLE_LINK = DATA_LINK + APPEND_TO_LINK

In [2]:
import pandas as pd

In [14]:
import requests
from bs4 import BeautifulSoup

html = requests.get(TABLE_LINK).text
soup = BeautifulSoup(html, 'lxml')
salas_cine = soup.find_all('table')[0]

In [17]:
type(salas_cine)

bs4.element.Tag

In [23]:
test_df = pd.read_html(str(salas_cine), skiprows=2, header=0)[0]

In [24]:
display(test_df.head())

,Ms ID #,Manuscript Title,Article Type,Corresponding Author Name,First Author Name,Corresponding Author Email,No. of supplementary files,Section,Unnamed: 8,Corresponding Author mailing address,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,paper_05,Gravity influence in one-dimensional blood flo...,research,NaN,NaN,NaN,https://github.com/ITMO-MMRM-lab/Complex_blood...,Healthcare,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,paper_06,Light-weight ensembling of deep neural models ...,research,NaN,NaN,NaN,NaN,Deep Learning and Data-Driven Modelling,36.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,paper_10,Comparative Analysis of SVM Classifiers in Cri...,research,NaN,NaN,NaN,NaN,Data Analysis and NLP,21.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,paper_12,"Hybrid Algorithm for Multi-Contractor, Multi-R...",research,A.S. Voskresenskii,A.S. Voskresenskii,NaN,NaN,Optimisation and Scheduling,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,paper_14,Framework architecture of a secure big data lake,research,NaN,NaN,NaN,https://github.com/IcyAltair/Pet-projects/tree...,Computing infrastructure,18.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df = pd.read_html(TABLE_LINK, skiprows=2, header=0)[0]

In [13]:
display(df.head())

,Ms ID #,Manuscript Title,Article Type,Corresponding Author Name,First Author Name,Corresponding Author Email,No. of supplementary files,Section,Unnamed: 8,Corresponding Author mailing address,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,paper_05,Gravity influence in one-dimensional blood flo...,research,NaN,NaN,NaN,https://github.com/ITMO-MMRM-lab/Complex_blood...,Healthcare,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,paper_06,Light-weight ensembling of deep neural models ...,research,NaN,NaN,NaN,NaN,Deep Learning and Data-Driven Modelling,36.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,paper_10,Comparative Analysis of SVM Classifiers in Cri...,research,NaN,NaN,NaN,NaN,Data Analysis and NLP,21.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,paper_12,"Hybrid Algorithm for Multi-Contractor, Multi-R...",research,A.S. Voskresenskii,A.S. Voskresenskii,NaN,NaN,Optimisation and Scheduling,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,paper_14,Framework architecture of a secure big data lake,research,NaN,NaN,NaN,https://github.com/IcyAltair/Pet-projects/tree...,Computing infrastructure,18.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
